In [1]:
# 트랜스포머 설치
!pip install transformers

In [2]:
# AdaBelief optimizer 설치
!pip install adabelief-pytorch==0.2.0

In [3]:
# library import
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import urllib.request

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [4]:
#random seed 설정
tf.random.set_seed(1234)
np.random.seed(1234)

#배치사이즈, 에폭 설정
BATCH_SIZE = 32
NUM_EPOCHS = 6

L_RATE = 1e-5

#입력데이터 최대 길이
MAX_LEN = 512

#Gradient Clipping -> L2-norm 이상 Gradient 벡터 정규화
max_grad_norm=1

#로그 출력 주기
log_interval=200

#CPU코어 개수
NUM_CORES = os.cpu_count()

#GPU 활용
device = torch.device("cuda:0")

DATA_IN_PATH = './data'
DATA_OUT_PATH = "./data"

## 금융 문장 데이터셋

In [5]:
# 깃허브 번역 금융 뉴스 기사 문장 데이터셋 경로
DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "finance_data.csv"

In [6]:
# 파일 다운로드
urllib.request.urlretrieve(DATASET_URL, filename = DATASET_NAME)

('finance_data.csv', <http.client.HTTPMessage at 0x7ce4f7b7e200>)

In [7]:
# 금융 뉴스 데이터셋
dataset = pd.read_csv(DATASET_NAME)
dataset.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [8]:
# 기존 영문 문장 제거
del dataset['sentence']

# 데이터셋 라벨데이터를 숫자로 치환
dataset['labels'] = dataset['labels'].replace(['neutral', 'positive', 'negative'],[0, 1, 2])

# 중복 데이터 제거
DATASET_PREP_FILE = 'dataset_prep.csv'
dataset.drop_duplicates(subset = ['kor_sentence'], inplace = True)
dataset.to_csv(DATASET_PREP_FILE)

## 학습 세팅
사용하고자 하는 klue/roberta-base 언어모델의 tokenizer를 불러와서 진행

In [9]:
# Klue-roberta-base model tokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base", cache_dir='bert_ckpt', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import pandas as pd
dataset = pd.read_csv('dataset_prep.csv')
dataset.drop('Unnamed: 0',axis=1,inplace=True)

In [11]:
# 입출력 데이터 분리
X_data = dataset['kor_sentence']
y_data = dataset['labels']

In [12]:
# train, test split
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = TEST_SIZE,
                                                    random_state = RANDOM_STATE,
                                                    stratify = y_data)

In [13]:
# 데이터셋으로 변환해주기 위해 형식을 맞추도록 하겠습니다.
X=pd.concat([X_train,y_train],axis=1)
X.reset_index(drop=True,inplace=True)
X_test=X_test.reset_index()

In [14]:
# 데이터셋 전처리
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # 문장 불러오기
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, # 문장 처음과 끝 토큰 추가
          max_length = MAX_LEN, #최대 길이를 설정
          pad_to_max_length = True, #길이에 따라 초과하는 부분 자르고,
          truncation=True, #모자란 부븐 패딩
          return_tensors="pt") #텐서로 반환

        padded_token_list = encoded_dict['input_ids'][0] #패딩된 토큰 id 리스트
        token_type_id = encoded_dict['token_type_ids'][0] #세그먼트 id -> 문장 구분
        att_mask = encoded_dict['attention_mask'][0] #마스킹 된 부분은 0, 실제 데이터셋이 있는 토큰 위치는 1
        target = torch.tensor(self.df_data.loc[index, "labels"]) # 타겟 텐서로 반환
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [15]:
# 동일한 방식 전처리
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True,
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [16]:
# 정확도 계산
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [17]:
#dataset -> 배치사이즈 만큼
train_data = TrainDataset(X)

test_data = TestDataset(X_test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=0)

#"klue/roberta-base" 모델 -> 중립(0), 긍정(1), 부정(2), label 분류
model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-base',num_labels=3)

#model GPU 할당
model.to(device)

#Optimizer 설정
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)


# 안정적인 학습을 위해 학습률을 점진적으로 증가시키고, 코사인 함수 형태로 감소하는 스케쥴러 설정 (전체 스텝의 10프로만)
warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad() #가중치 초기화
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1] #출력 로짓
        loss.backward() # 역전파
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) #gradient clipping
        optimizer.step()
        scheduler.step()
        _, preds = torch.max(out, dim=1)
        train_acc += f1_score(preds.cpu(), label.cpu(),average='weighted') #cpu로 이동, 샘플 가중치 주어 f1_score 계산
        if batch_id % log_interval == 0: # log_interval 마다 출력
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

preds = []

#평가 - 예측값 pres에 저장
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


<ipython-input-17-521531063dbb>:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/121 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

epoch 6 train acc 0.9664424881984603


<ipython-input-17-521531063dbb>:61: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/31 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [18]:
# 가장 로짓 높은 라벨
predicted_label = np.argmax(Preds, axis=1)

In [19]:
predicted_label

array([1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 0, 0, 2, 0, 2, 2, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 2, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 2, 2, 2, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0,
       2, 1, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 2,
       0, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2,
       0, 2, 1, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 1, 2, 1, 0, 2, 2, 0, 0, 1,
       0, 0, 2, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 0, 1, 1, 0, 0, 0, 1, 2, 0,
       0, 0, 0, 1, 1, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 2, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 1,

In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

# 사전학습 성능
path2data='./metric'
if not os.path.exists(path2data):
    os.mkdir(path2data)
CL_REPORT_FILE = "./metric/cl_report.csv"

cl_report = classification_report(y_test, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
cl_report_df.to_csv(CL_REPORT_FILE)
print(cl_report_df)

              precision  recall  f1-score  support
0                 0.898   0.890     0.894  572.000
1                 0.798   0.824     0.811  273.000
2                 0.872   0.843     0.857  121.000
accuracy          0.865   0.865     0.865    0.865
macro avg         0.856   0.852     0.854  966.000
weighted avg      0.866   0.865     0.866  966.000


In [21]:
# 모델 저장
torch.save(model.state_dict(), 'klue_base_fold6_s.pth')

In [22]:
# GPU
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Inference

In [23]:
# 매일경제 10개년치 전처리 완료된 데이터셋
import pandas as pd
final_매일경제 = pd.read_csv('/content/drive/MyDrive/BOAZ/23기 분석 BASE/미니프로젝트/감성분석/final_df.csv')

In [24]:
# 코랩 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
final_매일경제

,index,link,date,title,body,year,month,day,hour,minute,cleaned_title,cleaned_body
0,2015000001,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,CES 놀라게 할 Made In Korea,미국 라스베이거스에서 열리는 세계 최대 가전전시회 CES 2015를 깜짝 놀라게 할...,2015,1,1,4,3,놀라게 할.,미국 라스베이거스에서 열리는 세계 최대 가전 전시회 2015를 깜짝 놀라게 할 제품...
1,2015000002,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,삼성전자 바닥 치고 2분기부터 실적 개선될듯,"◆ 2015 업종별 유망주 ① IT·전자 ◆‘반도체 호황의 지속, 대형 패널 수요의...",2015,1,1,4,3,삼성전자 바닥 치고 2분기부터 실적 개선될 듯.,"반도체 호황의 지속, 대형 패널 수요의 증가, 스마트폰의 반등 전문가들이 말하는 새..."
2,2015000003,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,리서치센터장 5인이 보는 새해 증시 투자전략,"‘상저하고(上低下高·주가가 상반기에 부진하고 하반기에는 올라가는 것 ), 지배구조·...",2015,1,1,4,3,리서치센터장 5인이 보는 새해 증시 투자전략.,"상저하고 상저하고 주가가 상반기에 부진하고 하반기에는 올라가는 , 지배구조 배당 관..."
3,2015000005,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 15:31:00,삼성전자 타이젠 TV CES에서 공개,삼성전자는 타이젠 운영체제(OS)를 기반으로 한 스마트TV를 6일 미국 라스베이거스...,2015,1,1,15,31,삼성전자 타이젠 공개.,삼성전자는 타이젠 운영체제를 기반으로 한 스마트를 6일 미국 라스베이거스에서 열리는...
4,2015000006,https://m.entertain.naver.com/article/009/0003...,2015-01-01 17:07:00,[매경e신문] 오늘의 프리미엄,◆ PC시장서 재기 노리는 삼성전자삼성전자가 올해 초 ‘노트북9 2015년 에디션’...,2015,1,1,17,7,오늘의 프리미엄.,◆ 시장서 재기 노리는 삼성전자 삼성전자가 올해 초 노트북 9 2015년 에디션 올...
...,...,...,...,...,...,...,...,...,...,...,...,...
42911,2024003350,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 17:53:00,AI 특화 스마트폰 구글도 본격 뛰어든다,제미나이 탑재 픽셀9 공개애플보다 출시 두달 앞당겨OS동맹 삼성과 휴대폰 경쟁갤S2...,2024,8,14,17,53,특화 스마트폰 구글도 본격 뛰어든다.,제미나이 탑재 픽셀 9 공개 애플보다 출 시 두 달 앞당겨 동맹 삼성과 휴대폰 경쟁...
42912,2024003351,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:02:00,돌아온 엔비디아…반도체·소부장 '기지개',"美증시 AI관련주 강세에엔비디아 이달 17% 상승이턴 등 전력인프라도 '쑥'외국인,...",2024,8,14,18,2,돌아온 엔비디아 반도체 소부장 '기지개'.,"미 증시 관련주 강세에 엔비디아 달 17% 상승이 턴 전력 인프라도 '쑥' 외국인,..."
42913,2024003352,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:02:00,삼성전자 소액주주 1년새 142만명 줄었다,올들어 주가 급등락하자566만명서 424만명으로'국민주' 삼성전자의 소액주주가 45...,2024,8,14,18,2,삼성전자 소액주주 1년 새 142만명 줄었다.,올 들어 주가 급등락하자 566만명 서 424만명으로 '국민주' 삼성전자의 소액주주...
42914,2024003353,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:03:00,삼성전자 2분기 R&D에 8조 썼다,작년동기보다 12% 늘어 최대반도체 시설투자도 10조 달해삼성전자가 인공지능(AI)...,2024,8,14,18,3,삼성전자 2분기 8조 썼다.,작년 동기보다 12% 늘어 최대 반도체 시설 투자도 10조 달해 삼성전자가 인공지능...


In [26]:
# 전처리 완료 된 본문 불러온 후 column명 맞춰주기
result = final_매일경제['cleaned_body'].reset_index()
result.rename(columns={'cleaned_body' : 'kor_sentence'}, inplace = True)

In [28]:
# 동일 형태로 전처리
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'kor_sentence']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True,
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

# 추론 데이터 dataloader에 올리기
test_data = TestDataset(result)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=0)

In [29]:
# library import
import torch.cuda.amp as amp
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification,\
XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup


In [30]:
# 파라미터 설정
class args:
    debug=False
    amp = True
    gpu = '0'

    """
    epochs=5
    batch_size=32
    weight_decay=1e-6
    n_fold=5
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3

    exp_name = 'model_f'
    dir_ = f'./saved_models/'
    pt = 'mz_model'
    max_len = 128

    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None#'get_linear_schedule_with_warmup'
    """

# valid set 예측
def do_predict(net, valid_loader):

    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast(): #메모리 사용량 감소, 연산 속도 향상
                    # output
                    output = net(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)[0]

            else:
                output = net(outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask))

            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())

    return pred_lst,logit

# 현재 입력 데이터셋에 대해 추론
def run_predict(model_path,test_dataloader):


    print('set testloader')
    scaler = amp.GradScaler()

    net = RobertaForSequenceClassification.from_pretrained('klue/roberta-base', num_labels = 3)


    net.to(device)

    # gpu 여러개면 병렬 처리
    if len(args.gpu)>1:
        net = nn.DataParallel(net)

    f = torch.load(model_path) #가중치 저장된 모델 upload
    net.load_state_dict(f, strict=True)
    print('load saved models')

    preds, logit = do_predict(net, test_dataloader) #output

    print('complete predict')

    return preds, np.array(logit)

In [31]:
# 저장된 모델로 예측 수행
preds1, logit1 = run_predict('/content/drive/MyDrive/BOAZ/23기 분석 BASE/미니프로젝트/감성분석/klue_base_fold6_s.pth',test_dataloader)

set testloader


<ipython-input-30-f1615a308d5c>:60: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-30-f1615a308d5c>:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wil

load saved models


<ipython-input-30-f1615a308d5c>:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(valid_loader)):


  0%|          | 0/1342 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-30-f1615a308d5c>:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(): #메모리 사용량 감소, 연산 속도 향상


complete predict


In [32]:
# 예측 label
preds1

[1,
 1,
 2,
 0,
 2,
 2,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 1,
 1,


In [33]:
# 데이터셋 합쳐서 view
result['라벨링'] = preds1

In [34]:
result.head(50)

,index,kor_sentence,라벨링
0,0,미국 라스베이거스에서 열리는 세계 최대 가전 전시회 2015를 깜짝 놀라게 할 제품...,1
1,1,"반도체 호황의 지속, 대형 패널 수요의 증가, 스마트폰의 반등 전문가들이 말하는 새...",1
2,2,"상저하고 상저하고 주가가 상반기에 부진하고 하반기에는 올라가는 , 지배구조 배당 관...",2
3,3,삼성전자는 타이젠 운영체제를 기반으로 한 스마트를 6일 미국 라스베이거스에서 열리는...,0
4,4,◆ 시장서 재기 노리는 삼성전자 삼성전자가 올해 초 노트북 9 2015년 에디션 올...,2
5,5,"연말 연초를 맞아 재계가 임직원 희망퇴직을 받고. 새해에도 업황 부진에다 통상임금,...",2
6,6,삼성전자가 지난해 처음 애플을 꺾고 미국인이 가장 좋아하는 스마트폰 제조사 자리에 ...,1
7,7,역대 정부는 경제 활성화와 사회 통합을 위해 대통령 고유 권한인 사면 제도를 적절하...,1
8,8,배우 이정재가 대상그룹 회장 장녀 임세령과의 열애설에 대해 공식 입장을 밝혔다. 1...,0
9,9,삼성전자가 2일 서울 삼성전자 서초 사옥에서 권오현 대표이사 부회장을 비롯해 사장단...,1


In [35]:
# 기존 데이터셋에 label feature 추가
final_매일경제['label'] = preds1

In [36]:
final_매일경제

,index,link,date,title,body,year,month,day,hour,minute,cleaned_title,cleaned_body,label
0,2015000001,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,CES 놀라게 할 Made In Korea,미국 라스베이거스에서 열리는 세계 최대 가전전시회 CES 2015를 깜짝 놀라게 할...,2015,1,1,4,3,놀라게 할.,미국 라스베이거스에서 열리는 세계 최대 가전 전시회 2015를 깜짝 놀라게 할 제품...,1
1,2015000002,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,삼성전자 바닥 치고 2분기부터 실적 개선될듯,"◆ 2015 업종별 유망주 ① IT·전자 ◆‘반도체 호황의 지속, 대형 패널 수요의...",2015,1,1,4,3,삼성전자 바닥 치고 2분기부터 실적 개선될 듯.,"반도체 호황의 지속, 대형 패널 수요의 증가, 스마트폰의 반등 전문가들이 말하는 새...",1
2,2015000003,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 04:03:00,리서치센터장 5인이 보는 새해 증시 투자전략,"‘상저하고(上低下高·주가가 상반기에 부진하고 하반기에는 올라가는 것 ), 지배구조·...",2015,1,1,4,3,리서치센터장 5인이 보는 새해 증시 투자전략.,"상저하고 상저하고 주가가 상반기에 부진하고 하반기에는 올라가는 , 지배구조 배당 관...",2
3,2015000005,https://n.news.naver.com/mnews/article/009/000...,2015-01-01 15:31:00,삼성전자 타이젠 TV CES에서 공개,삼성전자는 타이젠 운영체제(OS)를 기반으로 한 스마트TV를 6일 미국 라스베이거스...,2015,1,1,15,31,삼성전자 타이젠 공개.,삼성전자는 타이젠 운영체제를 기반으로 한 스마트를 6일 미국 라스베이거스에서 열리는...,0
4,2015000006,https://m.entertain.naver.com/article/009/0003...,2015-01-01 17:07:00,[매경e신문] 오늘의 프리미엄,◆ PC시장서 재기 노리는 삼성전자삼성전자가 올해 초 ‘노트북9 2015년 에디션’...,2015,1,1,17,7,오늘의 프리미엄.,◆ 시장서 재기 노리는 삼성전자 삼성전자가 올해 초 노트북 9 2015년 에디션 올...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42911,2024003350,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 17:53:00,AI 특화 스마트폰 구글도 본격 뛰어든다,제미나이 탑재 픽셀9 공개애플보다 출시 두달 앞당겨OS동맹 삼성과 휴대폰 경쟁갤S2...,2024,8,14,17,53,특화 스마트폰 구글도 본격 뛰어든다.,제미나이 탑재 픽셀 9 공개 애플보다 출 시 두 달 앞당겨 동맹 삼성과 휴대폰 경쟁...,0
42912,2024003351,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:02:00,돌아온 엔비디아…반도체·소부장 '기지개',"美증시 AI관련주 강세에엔비디아 이달 17% 상승이턴 등 전력인프라도 '쑥'외국인,...",2024,8,14,18,2,돌아온 엔비디아 반도체 소부장 '기지개'.,"미 증시 관련주 강세에 엔비디아 달 17% 상승이 턴 전력 인프라도 '쑥' 외국인,...",1
42913,2024003352,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:02:00,삼성전자 소액주주 1년새 142만명 줄었다,올들어 주가 급등락하자566만명서 424만명으로'국민주' 삼성전자의 소액주주가 45...,2024,8,14,18,2,삼성전자 소액주주 1년 새 142만명 줄었다.,올 들어 주가 급등락하자 566만명 서 424만명으로 '국민주' 삼성전자의 소액주주...,2
42914,2024003353,https://n.news.naver.com/mnews/article/009/000...,2024-08-14 18:03:00,삼성전자 2분기 R&D에 8조 썼다,작년동기보다 12% 늘어 최대반도체 시설투자도 10조 달해삼성전자가 인공지능(AI)...,2024,8,14,18,3,삼성전자 2분기 8조 썼다.,작년 동기보다 12% 늘어 최대 반도체 시설 투자도 10조 달해 삼성전자가 인공지능...,1


In [37]:
# 최종 CSV Save
final_매일경제.to_csv('/content/drive/MyDrive/BOAZ/23기 분석 BASE/미니프로젝트/감성분석/final_매일경제.csv',encoding='utf-8-sig')